### Ejercicios del tema de Ensemble Learning y Random Forests

*Hugo Díaz Díaz* (*hdiazd00@estudiantes.unileon.es*)

*Correo profesional: hugo.didi.contacto@gmail.com*

---


## Parte teórica (opcional)

### 1. Si has entrenado cinco modelos diferentes con exactamente los mismos datos de entrenamiento, y todos logran una precisión del 95%, ¿hay alguna posibilidad de combinar estos modelos para obtener mejores resultados? Si es así, ¿cómo? Si no, ¿por qué?

Sí, se pueden combinar.
Si los cinco modelos no cometen exactamente los mismos errores, es decir, son lo suficientemente diversos en cuanto a algoritmo, hiperparámetros inicializaciones... un ensemble (por ejemplo, un clasificador de votación dura o suave) puede mejorar un poco la precisión respecto a cada modelo individual, porque promediar las predicciones reduce la varianza del sistema. Si fueran prácticamente idénticos, la mejora sería nula.

### 2. ¿Cuál es la diferencia entre clasificadores de voto duro y voto suave?

En voto duro cada modelo da una clase y la salida final es la clase más votada. En voto suave, cada modelo da probabilidades por clase, se promedian esas probabilidades y se elige la de mayor probabilidad media. El voto suave suele ir mejor porque tiene en cuenta el grado de confianza de los modelos.

### 3. ¿Es posible acelerar el entrenamiento de un ensemble de bagging distribuyéndolo entre múltiples servidores? ¿Qué pasa con los ensembles de pasting, boosting, random forests o stacking?

En bagging y pasting cada modelo se entrena de forma independiente en un subconjunto de datos, así que se pueden repartir entre varios núcleos/servidores sin problema. Lo mismo vale para los Random Forests, que son básicamente bagging de árboles. En boosting (AdaBoost, Gradient Boosting) los modelos se entrenan secuencialmente, cada uno corrige al anterior, así que no se puede paralelizar la secuencia. En stacking, los modelos base sí se pueden entrenar en paralelo, pero el meta-modelo va después con sus salidas.

### 4. ¿Cuál es el beneficio de la evaluación out-of-bag?

En bagging, cada modelo solo ve alrededor del 63 % de las instancias de entrenamiento, el resto se consideran out-of-bag para ese modelo. Eso permite usarlas como “test interno” sin tener que reservar un conjunto de validación separado, aprovechando todos los datos tanto para entrenar como para estimar el rendimiento del ensemble.

### 5. ¿Qué hace que los Extra-Trees sean más aleatorios que los Random Forests regulares? ¿Cómo puede ayudar esta aleatoriedad extra? ¿Son los Extra-Trees más lentos o más rápidos que los Random Forests regulares?

Los Extra-Trees son más aleatorios porque, además de muestrear características como en un Random Forest, eligen los umbrales de corte de forma totalmente aleatoria en cada nodo, en vez de buscar el mejor split. Esta aleatoriedad extra reduce la correlación entre árboles, baja la varianza (aunque suba un poco el sesgo) y suele mejorar la generalización. Al no buscar el mejor umbral, suelen ser más rápidos de entrenar que un Random Forest estándar.

### 6. Si tu ensemble de AdaBoost subajusta los datos de entrenamiento, ¿qué hiperparámetros deberías ajustar y cómo?

Si mi ensemble de **AdaBoost** está subajustando (no llega a aprender bien ni siquiera el conjunto de entrenamiento), lo que necesito es darle más capacidad al modelo. Desde el punto de vista de los hiperparámetros, las opciones típicas son:

- Aumentar `n_estimators`, es decir, añadir más clasificadores débiles al ensemble para que tenga más pasos de corrección de errores. 

- Hacer el estimador base algo más complejo (por ejemplo, árboles con mayor `max_depth` en lugar de simples decision stumps).

- Y, si sigo muy corto, se puede subir ligeramente `learning_rate`, porque una tasa de aprendizaje muy baja actúa como una regularización fuerte y puede estar frenando demasiado las correcciones.

### 7. Si tu ensemble de Gradient Boosting sobreajusta el conjunto de entrenamiento, ¿deberías aumentar o disminuir la tasa de aprendizaje?

En **Gradient Boosting**, el hiperparámetro `learning_rate` controla cuánto corrige cada nuevo árbol los errores de los anteriores. Un valor alto hace que cada árbol “empuje” mucho y es fácil que el modelo sobreajuste al conjunto de entrenamiento. Cuando veo overfitting, lo que debo hacer es disminuir la tasa de aprendizaje y, en compensación, permitir más árboles (`n_estimators` más grande). En las transparencias se comenta precisamente que valores bajos de `learning_rate` requieren más árboles pero tienden a generalizar mejor gracias a esta estrategia de *shrinkage*.

---

## Parte práctica (obligatoria)

### Ejercicio 8

Carga los datos MNIST (introducidos en el Capítulo 3) y divídelos en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, usa 50,000 instancias para entrenamiento, 10,000 para validación y 10,000 para prueba).

- Luego entrena varios clasificadores, como un clasificador Random Forest, un clasificador Extra-Trees y un SVM.
- A continuación, intenta combinarlos en un ensemble que supere a todos en el conjunto de validación, utilizando un clasificador de voto suave o duro.
- Una vez que hayas encontrado uno, pruébalo en el conjunto de prueba.
- ¿Cuánto mejor se desempeña en comparación con los clasificadores individuales?

Primero se cargan los datos de MNIST desde `fetch_openml`. Después se normalizan las imágenes y se realiza la partición en conjuntos de entrenamiento, validación y prueba con el esquema 50k/10k/10k.

In [18]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
import os
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

np.random.seed(42)

In [19]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)  # etiquetas como enteros

# Normalización sencilla [0, 1]
X = X / 255.0

# División 60k / 10k (como en el libro) y luego 50k / 10k / 10k
X_train_full, X_test = X[:60000], X[60000:]
y_train_full, y_test = y[:60000], y[60000:]

X_train, X_val = X_train_full[:50000], X_train_full[50000:]
y_train, y_val = y_train_full[:50000], y_train_full[50000:]

X_train.shape, X_val.shape, X_test.shape


((50000, 784), (10000, 784), (10000, 784))

En la siguiente celda se definen tres clasificadores distintos sobre el conjunto de entrenamiento: un Random Forest, un Extra-Trees y un SVM con kernel RBF. Para que el SVM sea computacionalmente manejable, se entrena sobre un subconjunto del conjunto de entrenamiento, mientras que la validación se hace siempre sobre las 10.000 instancias de validación.

In [20]:
# Modelos base: Random Forest, Extra-Trees y SVM
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

et_clf = ExtraTreesClassifier(
    n_estimators=100,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

# Para el SVM se usa un subconjunto del entrenamiento por coste computacional
svm_train_size = 10000
X_train_svm = X_train[:svm_train_size]
y_train_svm = y_train[:svm_train_size]

svm_clf = SVC(
    kernel="rbf",
    gamma="scale",
    C=10,
    probability=True,  # necesario para voto suave
    random_state=42
)


A continuación, se entrenan los modelos con sus respectivos datos de entrenamiento y se evalúan en el conjunto de validación, de forma que se tenga una referencia del rendimiento individual.

In [21]:
# Entrenamiento de los modelos individuales
rf_clf.fit(X_train, y_train)
et_clf.fit(X_train, y_train)
svm_clf.fit(X_train_svm, y_train_svm)

# Evaluación en el conjunto de validación
y_val_pred_rf = rf_clf.predict(X_val)
y_val_pred_et = et_clf.predict(X_val)
y_val_pred_svm = svm_clf.predict(X_val)

val_acc_rf = accuracy_score(y_val, y_val_pred_rf)
val_acc_et = accuracy_score(y_val, y_val_pred_et)
val_acc_svm = accuracy_score(y_val, y_val_pred_svm)

val_acc_rf, val_acc_et, val_acc_svm


(0.9734, 0.9743, 0.9706)

En este punto se dispone de las precisiones en validación de cada clasificador por separado. Los resultados obtenidos han sido aproximadamente 0.9734 para Random Forest, 0.9743 para Extra-Trees y 0.9706 para el SVM. Aunque Extra-Trees destaca ligeramente, las tres métricas son muy similares y ya bastante altas, lo que indica que todos los modelos capturan bien la estructura del problema. Aun así, es razonable pensar que sus errores no son totalmente coincidentes, especialmente entre los modelos basados en árboles y el SVM, por lo que la combinación de sus predicciones puede aportar una mejora adicional.

El siguiente paso consiste, por tanto, en construir un ensemble de votación que combine estos modelos. Se intentó utilizar un `VotingClassifier` con voto suave, ya que aprovecha las probabilidades de los modelos base y suele ofrecer un rendimiento algo superior. Sin embargo, dado que el SVM se reentrena internamente sobre las 50.000 instancias cuando se llama a fit() del VotingClassifier, el tiempo de cómputo se vuelve excesivo. Por ello, se opta por implementar manualmente el voto suave: se obtienen las probabilidades `predict_proba` de cada clasificador base, se promedian y se escoge la clase con mayor probabilidad media, reproduciendo el comportamiento del voto suave sin forzar un nuevo entrenamiento costoso del SVM.

In [28]:
def soft_voting_predict(models, X):
    """
    Realiza voto suave a partir de una lista de modelos que implementan predict_proba.
    Devuelve las clases predichas por el ensemble.
    """
    # Se acumulan las probabilidades de cada modelo
    proba_list = [clf.predict_proba(X) for clf in models]
    avg_proba = np.mean(proba_list, axis=0)
    # Se escoge la clase con mayor probabilidad media
    return np.argmax(avg_proba, axis=1)


A continuación se evalúa este ensemble de voto suave sobre el conjunto de validación y se compara su precisión con la de los clasificadores individuales.

In [29]:
# Ensemble de voto suave sobre el conjunto de validación
base_models = [rf_clf, et_clf, svm_clf]

y_val_pred_ensemble = soft_voting_predict(base_models, X_val)
val_acc_ensemble = accuracy_score(y_val, y_val_pred_ensemble)

val_acc_rf, val_acc_et, val_acc_svm, val_acc_ensemble


(0.9734, 0.9743, 0.9706, 0.9759)

En nuestro caso concreto, las precisiones en validación han sido 97,34% para Random Forest, 97,43% para Extra-Trees y 97,06% para el SVM, mientras que el ensemble de voto suave alcanza 97,59%. Es decir, el ensemble mejora ligeramente al mejor modelo individual (Extra-Trees) en unas dos décimas porcentuales, lo que confirma que los modelos aportan información complementaria y que la agregación de sus probabilidades permite corregir algunos errores que cometen por separado.

Una vez comprobado que el ensemble funciona bien en validación, el siguiente paso consiste en evaluar tanto los modelos individuales como el ensemble sobre el conjunto de prueba, usando exactamente los modelos ya entrenados.

In [30]:
# Evaluación de los modelos individuales en el conjunto de prueba
y_test_pred_rf = rf_clf.predict(X_test)
y_test_pred_et = et_clf.predict(X_test)
y_test_pred_svm = svm_clf.predict(X_test)

test_acc_rf = accuracy_score(y_test, y_test_pred_rf)
test_acc_et = accuracy_score(y_test, y_test_pred_et)
test_acc_svm = accuracy_score(y_test, y_test_pred_svm)

# Evaluación del ensemble de voto suave en el conjunto de prueba
y_test_pred_ensemble = soft_voting_predict(base_models, X_test)
test_acc_ensemble = accuracy_score(y_test, y_test_pred_ensemble)

test_acc_rf, test_acc_et, test_acc_svm, test_acc_ensemble


(0.968, 0.9703, 0.9684, 0.9721)

En los resultados obtenidos en el conjunto de prueba, las precisiones han sido 96,8% para Random Forest, 97,03% para Extra-Trees y 96,84% para el SVM, mientras que el ensemble de voto suave alcanza 97,21%. Es decir, el ensemble mejora ligeramente al mejor modelo individual (Extra-Trees) en unas pocas décimas, cumpliendo el objetivo del ejercicio: se consigue un rendimiento algo superior y un comportamiento más estable que cualquiera de los clasificadores por separado.

## Ejercicio 9

Ejecuta los clasificadores individuales del ejercicio anterior para hacer predicciones en el conjunto de validación, y crea un nuevo conjunto de entrenamiento con las predicciones resultantes:

- Cada instancia de entrenamiento es un vector que contiene el conjunto de predicciones de todos tus clasificadores para una imagen, y el objetivo es la clase de la imagen.
- Entrena un clasificador en este nuevo conjunto de entrenamiento. ¡Felicidades, acabas de entrenar un blender, y junto con los clasificadores forman un ensemble de stacking! Ahora evaluemos el ensemble en el conjunto de prueba.
- Para cada imagen en el conjunto de prueba, haz predicciones con todos tus clasificadores, luego alimenta las predicciones al blender para obtener las predicciones del ensemble.
- ¿Cómo se compara con el clasificador de votación que entrenaste anteriormente?